In [9]:
import gymnasium as gym
# from gym.wrappers import FlattenObservation
# from gym.utils.env_checker import check_env
# import tabletop
# import gym
import numpy as np
import tensorflow as tf
from tensorflow import keras as ks
# import gym
import matplotlib.pyplot as plt
import time
from tensorflow.keras import layers
# from gym.utils.play import play
# from Cope import debug
%load_ext autoreload
%autoreload 2
import SquarePacking.env
!pip install -e SquarePacking
from ReplayBuffer import ReplayBuffer

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Defaulting to user installation because normal site-packages is not writeable
Obtaining file:///home/leonard/hello/python/AI/SquarePacking/SquarePacking
  Preparing metadata (setup.py) ... done
  Attempting uninstall: SquarePacking
    Found existing installation: SquarePacking 0.0.1
    Uninstalling SquarePacking-0.0.1:
      Successfully uninstalled SquarePacking-0.0.1
  Running setup.py develop for SquarePacking


In [ ]:
# Actor-Critic Model from Online

# The inputs are in the obvervation space
num_inputs = 3*N
# We want to nudge each box a little in each parameter
num_outputs = num_inputs

inputs = layers.Input(shape=(num_inputs,))
common = layers.Dense(128, activation="relu")(inputs)
action = layers.Dense(num_outputs, activation="tanh")(common)
critic = layers.Dense(1)(common)

model = keras.Model(inputs=inputs, outputs=[action, critic])

optimizer = keras.optimizers.Adam(learning_rate=0.01)
huber_loss = keras.losses.Huber()
action_probs_history = []
critic_value_history = []
rewards_history = []
running_reward = 0
episode_count = 0

while True:  # Run until solved
    # obs, info = env.reset(seed=seed)
    obs, info = env.reset()
    episode_reward = 0

    with tf.GradientTape() as tape:
        for timestep in range(1, max_steps_per_episode):
            if GUI:
                env.render()

            # state = tf.convert_to_tensor(np.reshape(np.array((state['ant'], state['nearest food'], state['has food'])), (3,)))
            state = tf.convert_to_tensor(obs)
            state = tf.expand_dims(state, 0)

            # Predict action probabilities and estimated future rewards
            # from environment state
            action_probs, critic_value = model(state)
            critic_value_history.append(critic_value[0, 0])

            # Sample action from action probability distribution
            # I don't think this works, because 
            # action = np.random.choice(num_outputs, p=np.squeeze(action_probs))
            # action_probs_history.append(tf.math.log(action_probs[0, action]))
            action_probs_history.append(tf.math.log(action_probs))

            # Apply the sampled action in our environment
            state, reward, done, info = env.step(action_probs)
            rewards_history.append(reward)
            episode_reward += reward

            if done:
                break

        # Update running reward to check condition for solving
        running_reward = 0.05 * episode_reward + (1 - 0.05) * running_reward

        # Calculate expected value from rewards
        # - At each timestep what was the total reward received after that timestep
        # - Rewards in the past are discounted by multiplying them with gamma
        # - These are the labels for our critic
        returns = []
        discounted_sum = 0
        for r in rewards_history[::-1]:
            discounted_sum = r + patience * discounted_sum
            returns.insert(0, discounted_sum)

        # Normalize
        returns = np.array(returns)
        returns = (returns - np.mean(returns)) / (np.std(returns) + eps)
        returns = returns.tolist()

        # Calculating loss values to update our network
        history = zip(action_probs_history, critic_value_history, returns)
        actor_losses = []
        critic_losses = []
        for log_prob, value, ret in history:
            # At this point in history, the critic estimated that we would get a
            # total reward = `value` in the future. We took an action with log probability
            # of `log_prob` and ended up recieving a total reward = `ret`.
            # The actor must be updated so that it predicts an action that leads to
            # high rewards (compared to critic's estimate) with high probability.
            diff = ret - value
            actor_losses.append(-log_prob * diff)  # actor loss

            # The critic must be updated so that it predicts a better estimate of
            # the future rewards.
            critic_losses.append(
                huber_loss(tf.expand_dims(value, 0), tf.expand_dims(ret, 0))
            )

        # Backpropagation
        loss_value = sum(actor_losses) + sum(critic_losses)
        grads = tape.gradient(loss_value, model.trainable_variables)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))

        # Clear the loss and reward history
        action_probs_history.clear()
        critic_value_history.clear()
        rewards_history.clear()

    # Log details
    episode_count += 1
    if episode_count % 10 == 0:
        template = "running reward: {:.2f} at episode {}"
        print(f"running reward: {running_reward:.2f} at episode {episode_count}")

    # if running_reward > 195:  # Condition to consider the task solved
    if info['solved']:
        print(f"Solved at episode {episode_count}!")
        break


In [10]:
# Configuration & Enviorment

seed = 42
tf.random.set_seed(seed)
np.random.seed(seed)

# Set up the enviorment
env      = gym.make("SquarePacking/Square-v0", N=4, render_mode="shapely", shift_rate=.05, rot_rate=.05)
test_env = gym.make("SquarePacking/Square-v0", N=4, render_mode="shapely", shift_rate=.05, rot_rate=.05)

# Discount factor for past rewards. (Always between 0 and 1.)
patience = gamma = 0.99
# Number of epochs to run and train agent.
epochs = 100
# Number of steps of interaction (state-action pairs) for the agent and the environment in each epoch.
steps_per_epoch = 4000
# Maximum length of replay buffer.
replay_size = int(1e6)
# Interpolation factor in polyak averaging for target networks. Target networks are updated towards main networks according to:
# θ_target <- pθ_target + (1-p)θ
# where p is polyak. (Always between 0 and 1, usually close to 1.)
polyak = 0.995
# Learning rate for policy.
pi_lr = policy_lr = 1e-3
# Learning rate for Q-networks.
q_lr = Q_lr = 1e-3
# Minibatch size for SGD.
batch_size = 100
# Number of steps for uniform-random action selection, before running real policy. Helps exploration.
start_steps = 10000
# Number of env interactions to collect before starting to do gradient descent updates. Ensures replay buffer is full enough for useful updates.
update_after = 1000
# Number of env interactions that should elapse between gradient descent updates. Note: Regardless of how long
# you wait between updates, the ratio of env steps to gradient steps is locked to 1.
update_every = 50
# Stddev for Gaussian exploration noise added to policy at training time. (At test time, no noise is added.)
act_noise = 0.1
# Number of episodes to test the deterministic policy at the end of each epoch.
num_test_episodes = 10
# Maximum length of trajectory / episode / rollout.
max_ep_len = 1000
# The space dimentions
obs_dim = env.observation_space.shape[0]
act_dim = env.action_space.shape[0]
# Action limit for clamping: critically, assumes all dimensions share the same bound!
act_limit = env.action_space.high[0]
# Inputs to computation graph
# x_ph, a_ph, x2_ph, r_ph, d_ph = [tf.placeholder(dtype=tf.float32, shape=(None,dim) if dim else (None,)) for dim in (obs_dim, act_dim, obs_dim, None, None)]
# x_ph, a_ph, x2_ph, r_ph, d_ph = obs_dim, act_dim, obs_dim, None, None
# Experience buffer
replay_buffer = ReplayBuffer(obs_dim=obs_dim, act_dim=act_dim, size=replay_size)

# How often (in terms of gap between epochs) to save the current policy and value function.
# save_freq = 1
# Any kwargs appropriate for the actor_critic function you provided to DDPG.
# ac_kwargs = dict(hidden_sizes=[args.hid]*args.l)
# Share information about action space with policy architecture
# ac_kwargs['action_space'] = env.action_space


#? Don't know what this means
# eps = np.finfo(np.float32).eps.item()  # Smallest number such that 1.0 + eps != 1.0

/home/leonard/.local/lib/python3.10/site-packages/gymnasium/spaces/box.py:129: UserWarning: WARN: Box bound precision lowered by casting to float32
  gym.logger.warn(f"Box bound precision lowered by casting to {self.dtype}")
/home/leonard/.local/lib/python3.10/site-packages/gymnasium/utils/passive_env_checker.py:35: UserWarning: WARN: A Box observation space has an unconventional shape (neither an image, nor a 1D vector). We recommend flattening the observation to have only a 1D vector or use a custom policy to properly process the data. Actual observation shape: (4, 3)
  logger.warn(


In [8]:
# Main outputs from computation graph

# Actor-critic functions

def actor_critic():
    """ A function which takes in placeholder symbols for state, ``x_ph``, and action, ``a_ph``, 
    and returns the main outputs from the agent's Tensorflow computation graph:
    ===========  ================  ======================================
    Symbol       Shape             Description
    ===========  ================  ======================================
    ``pi``       (batch, act_dim)  | Deterministically computes actions
                                   | from policy given states.
    ``q``        (batch,)          | Gives the current estimate of Q* for
                                   | states in ``x_ph`` and actions in
                                   | ``a_ph``.
    ``q_pi``     (batch,)          | Gives the composition of ``q`` and
                                   | ``pi`` for states in ``x_ph``:
                                   | q(x, pi(x)).
    ===========  ================  ======================================
    """

    def mlp(size, hidden_sizes=(256, 256)):
        for h in hidden_sizes[:-1]:
            unknownVar = tf.layers.Dense(size, units=h, activation='relu')
        return tf.layers.Dense(unknownVar, units=hidden_sizes[-1], activation='tanh')

    _act_dim = act_dim.shape.as_list()[-1]
    _act_limit = env.action_space.high[0]

    pi = act_limit * mlp(list(hidden_sizes)+[act_dim])
    q = tf.squeeze(mlp(tf.concat([obs_dim, act_dim], axis=-1), list(hidden_sizes)+[1], tf.nn.relu, None), axis=1)
    q_pi = tf.squeeze(mlp(tf.concat([obs_dim, pi], axis=-1), list(hidden_sizes)+[1], tf.nn.relu, None), axis=1)

    return pi, q, q_pi


# with tf.variable_scope('main'):
pi, q, q_pi = actor_critic(obs_dim, act_dim, **ac_kwargs)

# Target networks
# with tf.variable_scope('target'):
# Note that the action placeholder going to actor_critic here is irrelevant, because we only need q_targ(s, pi_targ(s)).
# ac_kwargs = dict(hidden_sizes=[args.hid]*args.l)
# ac_kwargs['action_space'] = env.action_space
pi_targ, _, q_pi_targ  = actor_critic(obs_dim, act_dim, hidden_sizes=[args.hid]*args.l, action_space=)

# Count variables
# var_counts = tuple(core.count_vars(scope) for scope in ['main/pi', 'main/q', 'main'])
# print('\nNumber of parameters: \t pi: %d, \t q: %d, \t total: %d\n'%var_counts)


# Bellman backup for Q function
# backup = tf.stop_gradient(r_ph + patience*(1-d_ph)*q_pi_targ)
backup = tf.stop_gradient(r_ph + patience*(1-int(done))*q_pi_targ)

# DDPG losses
pi_loss = -tf.reduce_mean(q_pi)
q_loss = tf.reduce_mean((q-backup)**2)

# Separate train ops for pi, q
pi_optimizer = tf.train.AdamOptimizer(learning_rate=pi_lr)
q_optimizer = tf.train.AdamOptimizer(learning_rate=q_lr)
train_pi_op = pi_optimizer.minimize(pi_loss, var_list=get_vars('main/pi'))
train_q_op = q_optimizer.minimize(q_loss, var_list=get_vars('main/q'))

# Polyak averaging for target variables
target_update = tf.group([tf.assign(v_targ, polyak*v_targ + (1-polyak)*v_main)
                            for v_main, v_targ in zip(get_vars('main'), get_vars('target'))])

# Initializing targets to match main variables
target_init = tf.group([tf.assign(v_targ, v_main)
                            for v_main, v_targ in zip(get_vars('main'), get_vars('target'))])

sess = tf.Session()
sess.run(tf.global_variables_initializer())
sess.run(target_init)

# Prepare for interaction with environment
total_steps = steps_per_epoch * epochs
start_time = time.time()
o, ep_ret, ep_len = env.reset(), 0, 0



NameError: name 'ac_kwargs' is not defined

In [ ]:

def get_action(obs, noise_scale):
    action = sess.run(pi, feed_dict={x_ph: obs.reshape(1,-1)})[0]
    action += noise_scale * np.random.randn(act_dim)
    return np.clip(action, -act_limit, act_limit)

def test_agent():
    for j in range(num_test_episodes):
        obs, done, ep_ret, ep_len = test_env.reset(), False, 0, 0
        while not(done or (ep_len == max_ep_len)):
            # Take deterministic actions at test time (noise_scale=0)
            obs, reward, done, _ = test_env.step(get_action(obs, 0))
            ep_ret += reward
            ep_len += 1
        # logger.store(TestEpRet=ep_ret, TestEpLen=ep_len)



In [ ]:
# Main loop: collect experience in env and update/log each epoch
for t in range(total_steps):
    # Until start_steps have elapsed, randomly sample actions
    # from a uniform distribution for better exploration. Afterwards,
    # use the learned policy (with some noise, via act_noise).
    if t > start_steps:
        action = get_action(o, act_noise)
    else:
        action = env.action_space.sample()

    # Step the env
    obs2, reward, done, _ = env.step(a)
    ep_ret += reward
    ep_len += 1

    # Ignore the "done" signal if it comes from hitting the time
    # horizon (that is, when it's an artificial terminal signal
    # that isn't based on the agent's state)
    done = False if ep_len == max_ep_len else done

    # Store experience to replay buffer
    replay_buffer.store(obs, action, reward, obs2, done)

    # Super critical, easy to overlook step: make sure to update
    # most recent observation!
    obs = obs2

    # End of trajectory handling
    if done or (ep_len == max_ep_len):
        # logger.store(EpRet=ep_ret, EpLen=ep_len)
        obs, ep_ret, ep_len = env.reset(), 0, 0

    # Update handling
    if t >= update_after and t % update_every == 0:
        for _ in range(update_every):
            batch = replay_buffer.sample_batch(batch_size)
            feed_dict = {x_ph: batch['obs1'],
                            x2_ph: batch['obs2'],
                            a_ph: batch['acts'],
                            r_ph: batch['rews'],
                            d_ph: batch['done']
                        }

            # Q-learning update
            outs = sess.run([q_loss, q, train_q_op], feed_dict)
            # logger.store(LossQ=outs[0], QVals=outs[1])

            # Policy update
            outs = sess.run([pi_loss, train_pi_op, target_update], feed_dict)
            # logger.store(LossPi=outs[0])

    # End of epoch wrap-up
    if (t+1) % steps_per_epoch == 0:
        epoch = (t+1) // steps_per_epoch

        # Save model
        # if (epoch % save_freq == 0) or (epoch == epochs):
        #     logger.save_state({'env': env}, None)

        # Test the performance of the deterministic version of the agent.
        test_agent()
